In [10]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import fft
from scipy import linalg as la
from ImgProcess import *     # import all functions from ../Codes/ImgProcessing.py
img = plt.imread('../Pictures/lena.tif')
size = 16
patches, locs, dcs = Img2patch(img, size, 4)
mD = fft.dct(np.eye(2 * size**2), norm='ortho')[: size**2]

In [39]:
OMP(patches, mD, 4)

array([[  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
        -54.65156138,  65.44947389],
       ...,
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])